<a href="https://colab.research.google.com/github/dasd412/my-first-ai/blob/main/feature_engineering_and_regularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 특성 공학 : 기존의 특성을 사용해 새로운 특성을 뽑아내는 작업
# (ex) 농어 길이 x 농어 높이를 새로운 특성으로 만들기
# 선형 회귀는 특성이 많을수록 엄청난 효과를 낸다. (매우 복잡한 모델을 표현할 수 있음)
# 여러 개의 특성을 사용한 선형 회귀를 '다중 회귀'라고 한다.

In [ ]:
import pandas as pd

# read_csv()에 url을 넣은 후, 데이터 프레임을 만든다.
df=pd.read_csv('https://bit.ly/perch_csv_data')
# 데이터 프레임을 넘파이 배열로 바꾼다.
perch_full=df.to_numpy()

print(perch_full)

In [ ]:
import numpy as np

# 타깃 데이터
perch_weight = np.array([5.9, 32.0, 40.0, 51.5, 70.0, 100.0, 78.0, 80.0, 85.0, 85.0, 110.0,
       115.0, 125.0, 130.0, 120.0, 120.0, 130.0, 135.0, 110.0, 130.0,
       150.0, 145.0, 150.0, 170.0, 225.0, 145.0, 188.0, 180.0, 197.0,
       218.0, 300.0, 260.0, 265.0, 250.0, 250.0, 300.0, 320.0, 514.0,
       556.0, 840.0, 685.0, 700.0, 700.0, 690.0, 900.0, 650.0, 820.0,
       850.0, 900.0, 1015.0, 820.0, 1100.0, 1000.0, 1100.0, 1000.0,
       1000.0])


In [ ]:
from sklearn.model_selection import train_test_split

train_input, test_input,train_target,test_target=train_test_split(perch_full,perch_weight,random_state=42)

In [ ]:
# 사이킷런의 변환기 : 특성을 만들거나 전처리하기 위한 클래스들
from sklearn.preprocessing import PolynomialFeatures

# 예제로, 2개의 특성 2와 3으로 이루어진 샘플 하나 적용해보기
poly=PolynomialFeatures()
poly.fit([[2,3]]) # transform() 전에 fit()을 해야 한다!!
print(poly.transform([[2,3]]))

In [ ]:
#fit()은 새롭게 만들 특성 조합을 찾고, transform()은 실제로 데이터를 변환한다.
# 그리고 변환기는 타깃 데이터 없이, 입력 데이터를 변환해준다.
# PolynomialFeatures 는 기본적으로 각 특성을 제곱한 항을 추가하고 특성끼리 서로 곱한 항을 추가한다.

poly=PolynomialFeatures(include_bias=False) # include_bias = False 면 특성 1이 나오지 않게 된다. (절편 값 무시)
poly.fit([[2,3]])
print(poly.transform([[2,3]]))


In [ ]:
poly=PolynomialFeatures(include_bias=False)

poly.fit(train_input)
train_poly=poly.transform(train_input)
print(train_poly.shape) # (데이터의 개수, 특성 개수)

In [ ]:
# 9개의 특성이 각각 어떤 입력의 조합으로 만들어졌는지 알려주는 메서드
poly.get_feature_names_out()

In [ ]:
# 테스트 세트도 변환
test_poly=poly.transform(test_input)

In [ ]:
# 다중 회귀는 여러 개의 특성을 사용하여 선형 회귀를 수행하는 것이기 때문에, 선형 회귀 모델을 훈련하는 것과 같은 방식으로 훈련한다.
from sklearn.linear_model import LinearRegression

lr=LinearRegression()
lr.fit(train_poly,train_target)
print(lr.score(train_poly,train_target)) # 특성이 늘어나면, 선형 회귀 예측 점수가 좋아짐을 확인함.

In [ ]:
print(lr.score(test_poly,test_target))

In [ ]:
#PolynomialFeatures()의 degree 매개 변수를 활용하면, 필요한 고차항의 최대 차수를 지정할 수  있다.
poly=PolynomialFeatures(degree=5,include_bias=False)# 5제곱 까지 특성을 만들어서 특성을 더 추가해본다.

poly.fit(train_input)
train_poly=poly.transform(train_input)
test_poly=poly.transform(test_input)

print(train_poly.shape) # 만들어진 특성이 55개나 된다.

In [ ]:
lr.fit(train_poly,train_target)
print(lr.score(train_poly,train_target)) # 훈련 점수는 더 좋아졌다.

In [ ]:
print(lr.score(test_poly,test_target)) # 값이 음수가 나온다!!
# 원인 : 특성의 개수를 크게 늘리면 선형 모델은 강력해진다. 그래서 훈련 세트에 대해선 거의 완벽하게 학습할 수 있다.
# 하지만, 이런 모델은 훈련 세트에 너무 과대 적합되므로 테스트 세트에서는 점수가 매우 낮아진다.

In [ ]:
# 규제(regularization)는 머신 러닝 모델이 훈련 세트를 너무 과도하게 학습하지 못하게 훼방하는 장치이다.
# 선형 회귀 모델에서는 특성에 곱해지는 계수(기울기)의 크기를 작게 만든다.

# 그런데 특성의 스케일이 정규화되지 않으면, 여기에 곱해지는 계수 값도 차이가 나게 된다. 그래서 선형 회귀 모델에 규제를 적용할 때 계수 값의 크기가
# 서로 많이 다르면 공정하게 제어되지 않는다. 즉, 규제를 적용하기 전에 먼저 정규화를 진행한다.

from sklearn.preprocessing import StandardScaler

ss=StandardScaler()
ss.fit(train_poly)#훈련 세트로 학습
train_scaled=ss.transform(train_poly)
test_scaled=ss.transform(test_poly) # 반드시 훈련 세트로 학습한 변환기를 사용해서 테스트 세트까지 변환되어야만 한다!!

In [ ]:
# 선형 회귀 모델에 규제를 추가한 모델을 릿지와 라쏘라고 한다.
# 릿지 : 계수를 제곱한 값을 기준으로 규제를 적용
# 라쏘 : 계수의 절댓값을 기준으로 규제를 적용 (크기를 아예 0으로 만들 수 도 있음)

In [ ]:
from sklearn.linear_model import Ridge

ridge=Ridge()
ridge.fit(train_scaled,train_target)
print(ridge.score(train_scaled,train_target))

In [ ]:
print(ridge.score(test_scaled,test_target)) # 규제를 적용한 결과, 훈련 세트에 과대 적합했던 문제를 해결하였음.

In [ ]:
# 릿지와 라쏘 모델을 사용할 때, 규제의 양을 임의로 조정할 수 있다.
# 모델 객체를 만들 때 alpha 매개변수로 규제의 강도를 조절한다. alpha 값이 커지면 규제 강도가 세지므로 계수값을 더 줄이고 조금 더 과소적합되도록 유도한다.
# alpha 값이 작으면 계수를 줄이는 역할이 줄어들고 선형회귀 모델과 유사해지므로 과대적합될 가능성이 커진다.

# alpha 값과 같이, 모델이 학습하는 값이 아니라 사전에 사람이 직접 지정해야 하는 매개변수를 '하이퍼 파라미터'라고 한다.

In [ ]:
# 적절한 alpha 값을 찾는 방법 중 하나는 alpha 값에 대한 R^2 그래프를 그려보는 것이다.
# 훈련 세트와 테스트 세트의 점수가 가장 가까운 지점이 최적의 alpha 값이 된다.

import matplotlib.pyplot as plt

train_score=[]
test_score=[]


In [ ]:
alpha_list=[0.001, 0.01, 0.1,1,10,100]

for alpha in alpha_list:
    # 릿지 모델을 만든다.
    ridge=Ridge(alpha=alpha)
    #릿지 모델을 훈련시킨다.
    ridge.fit(train_scaled,train_target)

    # 훈련 세트와 테스트 세트 점수를 저장한다.
    train_score.append(ridge.score(train_scaled,train_target))
    test_score.append(ridge.score(test_scaled,test_target))

In [ ]:
plt.plot(np.log10(alpha_list),train_score) # alpha 값이 10배씩 차이나므로 로그를 씌워서 간격을 비슷하게 한다.
plt.plot(np.log10(alpha_list),test_score)

# 위 그래프가 훈련 세트, 아래 그래프가 테스트 테스트를 나타낸다.
plt.xlabel('alpha')
plt.ylabel('R^2')
plt.show()

# alpha 값 10^-1=0.1이 훈련 세트와 테스트 세트의 점수가 가장 가까운 지점이므로 최적의 alpha 값이 된다.
# 그 왼쪽은 과대적합을, 그 오른쪽은 과소적합 경향을 보이고 있다.

In [ ]:
#최적의 alpha 값을 기준으로 규제를 적용해서 훈련
ridge=Ridge(alpha=0.1)
ridge.fit(train_scaled,train_target)

print(ridge.score(train_scaled,train_target))
print(ridge.score(test_scaled,test_target))


In [ ]:
#라쏘 모델 훈련해보기
from sklearn.linear_model import Lasso
lasso=Lasso()
lasso.fit(train_scaled,train_target)
print(lasso.score(train_scaled,train_target))

In [ ]:
print(lasso.score(test_scaled,test_target))

In [ ]:
train_score=[]
test_score=[]

alpha_list=[0.001, 0.01, 0.1,1,10,100]

for alpha in alpha_list:
    # 라쏘 모델 만들기
    lasso=Lasso(alpha=alpha, max_iter=10000)

    lasso.fit(train_scaled,train_target)

    train_score.append(lasso.score(train_scaled,train_target))
    test_score.append(lasso.score(test_scaled,test_target))

In [ ]:
plt.plot(np.log10(alpha_list),train_score)
plt.plot(np.log10(alpha_list),test_score)

plt.xlabel('alpha')
plt.ylabel('R^2')
plt.show()
# 위쪽이 훈련 세트 그래프, 아래쪽이 테스트 세트 그래프. 이 모델에서 가장 적합한 alpha 값은 1, 10^1=10이다.

In [ ]:
lasso=Lasso(alpha=10)

lasso.fit(train_scaled,train_target)

print(lasso.score(train_scaled,train_target))
print(lasso.score(test_scaled,test_target))

In [ ]:
#라쏘 모델은 계수값을 아예 0으로 만들 수도 있다. 다음은 0이 된 값의 개수를 보여준다.
print(np.sum(lasso.coef_==0))